#### Моделирование:
- Использовать веса при обучении или семплировании
$ w = (tr - b), b \space is \space budget $
- Использовать GAM, GLM, Poly

> Исходя из предположений о положительной корреляционной зависимости между бюджетом на кампанию и total revenue ...

In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import pickle

## Feature engineering

In [3]:
company = pd.read_csv('data/company.csv', index_col='id')
product = pd.read_csv('data/product.csv', index_col='id')
user = pd.read_csv('data/user.csv', index_col='id')
asset = pd.read_csv('data/asset.csv', index_col='id')
AU = pd.read_csv('data/AssetUser.csv', index_col='id')
AUG = pd.read_csv('data/AssetUserGroup.csv')
campaign = pd.read_csv('data/campaign.csv', index_col='id')

In [4]:
campaign

,product,AUG,rounds,budget,revenue,start,design,photos,movie,site,accounts
id,,,,,,,,,,,
0,0,906,4,27000.0,25000.0,2022-10-24,0,1500,0,3000,0
1,1,590,1,13000.0,25000.0,2022-05-15,0,0,0,0,0
2,2,541,2,22000.0,25000.0,2022-09-18,0,0,0,0,0
3,3,714,3,13000.0,25000.0,2022-07-06,0,0,0,0,0
4,4,1139,3,24000.0,25000.0,2022-11-03,0,1500,0,0,800
...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,792,1,18000.0,25000.0,2022-04-01,0,0,0,3000,0
7996,7996,830,2,24000.0,25000.0,2022-08-05,0,0,0,0,800
7997,7997,847,2,36000.0,29000.0,2022-10-20,0,0,0,0,0


In [5]:
product.columns

Index(['company', 'price', 'novelty', 'uniqueness', 'conversion', 'popularity',
       'description', 'volume', 'type'],
      dtype='object')

In [6]:
product_cols = ['price', 'novelty', 'uniqueness', 'conversion', 'popularity', 'volume', 'type']
campaign[product_cols] = product.loc[campaign['product'], product_cols]

In [7]:
AUG_ = AUG.loc[campaign.AUG, :].drop('price', axis=1)

In [8]:
campaign['AU_activity'] = (AUG_ * AU.loc[:, 'activity'].values.T).sum(1).values
campaign['AU_conversion'] = (AUG_ * AU.loc[:, 'conversion'].values.T).sum(1).values

In [9]:
campaign['company_popularity'] = company.loc[product.loc[campaign['product'], 'company'], 'popularity'].values

In [10]:
campaign['U_conversion'] = (AUG_ * user.loc[AU.loc[:, 'user'], 'conversion'].values.T).sum(1).values
campaign['U_activity'] = (AUG_ * user.loc[AU.loc[:, 'user'], 'activity'].values.T).sum(1).values

In [11]:
campaign

,product,AUG,rounds,budget,revenue,start,design,photos,movie,site,...,uniqueness,conversion,popularity,volume,type,AU_activity,AU_conversion,company_popularity,U_conversion,U_activity
id,,,,,,,,,,,,,,,,,,,,,
0,0,906,4,27000.0,25000.0,2022-10-24,0,1500,0,3000,...,0.003,0.126,0.451,1.0,service,12.325,13.964,0.271,5.588,17.276
1,1,590,1,13000.0,25000.0,2022-05-15,0,0,0,0,...,0.007,0.227,0.657,1.0,souvenirs,15.176,16.557,0.053,7.372,21.309
2,2,541,2,22000.0,25000.0,2022-09-18,0,0,0,0,...,0.136,0.212,0.278,1.0,toys,24.351,26.114,0.070,11.632,33.598
3,3,714,3,13000.0,25000.0,2022-07-06,0,0,0,0,...,0.007,0.009,0.170,1.0,clothes,14.698,14.328,0.137,6.599,18.895
4,4,1139,3,24000.0,25000.0,2022-11-03,0,1500,0,0,...,0.001,0.114,0.249,1.0,stickers,11.002,12.979,0.121,4.769,15.271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,7995,792,1,18000.0,25000.0,2022-04-01,0,0,0,3000,...,0.031,0.262,0.075,1.0,stuff,18.667,19.061,0.101,8.531,26.093
7996,7996,830,2,24000.0,25000.0,2022-08-05,0,0,0,0,...,0.007,0.041,0.236,1.0,service,25.973,30.825,0.072,12.580,34.085
7997,7997,847,2,36000.0,29000.0,2022-10-20,0,0,0,0,...,0.013,0.184,0.325,74.0,toys,39.327,43.932,1.000,17.942,54.356


In [12]:
campaign.columns

Index(['product', 'AUG', 'rounds', 'budget', 'revenue', 'start', 'design',
       'photos', 'movie', 'site', 'accounts', 'price', 'novelty', 'uniqueness',
       'conversion', 'popularity', 'volume', 'type', 'AU_activity',
       'AU_conversion', 'company_popularity', 'U_conversion', 'U_activity'],
      dtype='object')

In [13]:
features = [
    'rounds', 'novelty', 'uniqueness', 'conversion', 'popularity', 'volume', 'AU_activity',
    'AU_conversion', 'price', 'company_popularity', 'U_conversion', 'U_activity'
]
X, y = campaign[features], campaign['revenue']

## Modeling

### Baseline

In [14]:
from sklearn.linear_model import Ridge as Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import PolynomialFeatures

In [15]:
poly = PolynomialFeatures(degree=4)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [17]:
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)

In [18]:
poly.n_output_features_

1820

In [19]:
model = Model(alpha=2)

In [20]:
model.fit(X_train, y_train)

Ridge(alpha=2)

In [21]:
MSE(model.predict(X_test), y_test, squared=False) ## RMSE

902.1591530582139

In [22]:
model.score(X_test, y_test) ## R2 score

0.9070376996047949

In [23]:
tmp = pd.DataFrame(np.vstack([y_test, model.predict(X_test)]).T)
px.scatter(tmp, template='plotly_dark')

In [24]:
with open('baseline.pickle', 'wb') as file:
    pickle.dump(model, file)